In [1]:
from google.colab import files
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Bot_sides')

Mounted at /content/drive


In [2]:
!pip install tflearn

     |████████████████████████████████| 112kB 8.3MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp36-none-any.whl size=127301 sha256=6059c897a43b6c55655818ce218b70450ca460a493e7efa1b4ab0065909dd5b2
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [3]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

import json
with open('intents.json') as file:
    data = json.load(file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
words = []
labels = []
docs_x = []
docs_y = []

In [5]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [6]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [7]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [8]:
training = numpy.array(training)
output = numpy.array(output)

In [9]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")


Training Step: 21999  | total loss: 0.00004 | time: 0.131s
| Adam | epoch: 1000 | loss: 0.00004 - acc: 1.0000 -- iter: 168/175
Training Step: 22000  | total loss: 0.00004 | time: 0.138s
| Adam | epoch: 1000 | loss: 0.00004 - acc: 1.0000 -- iter: 175/175
--
INFO:tensorflow:/content/drive/My Drive/Bot_sides/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        print(tag)

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: Hello
greeting
Hello! super excited to know you more :)
You: What is your name
name
Just call me Maya. Maya means affection in Bangla language in case you are willing to know what that means. I know I am a mere bot, but I really want to show affection to my users, or at least I will try my best to do so :) 
You: What do you do
about
I am Maya. I want to talk to you to learn more about your mental state so that I can help you out. I want you to feel uplifted and good. It will be great for me if you are a little detailed while talking. I am being consistently updated and at some point, I will hopefully be able to extract meaningful information from even an undetailed conversation just like a human being,  but for the time being thanks for your consideration :) If you want to allow me to know your mental state, type “help” 
You: help
help
Thanks so much for allowing me to know you more :) I will ask you some questions about how you are 